### Experimenting with scipy's sparse matrices.

Shall explore scipy sparse matrices and how to use them to implement ALS algorithms. ALS is used for collaborative filtering algorithms which primarily rely on explicit ratings. The explicit rating matrices are user-item rating matrix which can be heavily sparse.

### Compressed sparse row (CSR, CRS or Yale format)[[source]](https://en.wikipedia.org/wiki/Sparse_matrix#Compressed_sparse_row_(CSR,_CRS_or_Yale_format))

The _compressed sparse row_ (CSR) or _compressed row storage_ (CRS) or Yale format represents a matrix **M** by three (one-dimensional) arrays, that respectively contain nonzero values, the extents of rows, and column indices. It is similar to COO, but compresses the row indices, hence the name. This format allows fast row access and matrix-vector multiplications (**M**_x_). The CSR format has been in use since at least the mid-1960s, with the first complete description appearing in 1967.

The CSR format stores a sparse _m_ × _n_ matrix **M** in row form using three (one-dimensional) arrays (V, COL\_INDEX, ROW\_INDEX). Let NNZ denote the number of nonzero entries in **M**. (Note that [zero-based indices](/wiki/Zero-based_numbering "Zero-based numbering") shall be used here.)

*   The arrays V and COL\_INDEX are of length NNZ, and contain the non-zero values and the column indices of those values respectively
*   COL\_INDEX contains the column in which the corresponding entry V is located.
*   The array ROW\_INDEX is of length _m_ + _1_ and encodes the index in V and COL\_INDEX where the given row starts. This is equivalent to ROW\_INDEX\[j\] encoding the total number of nonzeros above row j. The last element is NNZ , i.e., the fictitious index in V immediately after the last valid index NNZ - 1.[\[8\]](#cite_note-8)

For example, the matrix

![{\displaystyle {\begin{pmatrix}5&0&0&0\\0&8&0&0\\0&0&3&0\\0&6&0&0\\\end{pmatrix}}}](https://wikimedia.org/api/rest_v1/media/math/render/svg/d3ad9e3299e6c5619c049d4518f82bad916960e6)

is a 4 × 4 matrix with 4 nonzero elements, hence

```
V         = [ 5 8 3 6]
COL_INDEX = [ 0 1 2 1]
ROW_INDEX = [ 0 1 2 3 4]
```

assuming a zero-indexed language.

To extract a row, we first define:

row\_start = ROW\_INDEX\[row\]
row\_end   = ROW\_INDEX\[row + 1\]

Then we take slices from V and COL\_INDEX starting at row\_start and ending at row\_end.

To extract the row 1 (the second row) of this matrix we set `row_start=1` and `row_end=2`. Then we make the slices `V[1:2] = [8]` and `COL_INDEX[1:2] = [1]`. We now know that in row 1 we have one element at column 1 with value 8.

In this case the CSR representation contains 13 entries, compared to 16 in the original matrix. The CSR format saves on memory only when NNZ < (_m_ (_n_ − 1) − 1) / 2.

Another example, the matrix

( 10 20 0 0 0 0 0 30 0 40 0 0 0 0 50 60 70 0 0 0 0 0 0 80 ) {\\displaystyle {\\begin{pmatrix}10&20&0&0&0&0\\\\0&30&0&40&0&0\\\\0&0&50&60&70&0\\\\0&0&0&0&0&80\\\\\\end{pmatrix}}}

![{\displaystyle {\begin{pmatrix}10&20&0&0&0&0\\0&30&0&40&0&0\\0&0&50&60&70&0\\0&0&0&0&0&80\\\end{pmatrix}}}](https://wikimedia.org/api/rest_v1/media/math/render/svg/80a67bed613b4fe0f0d8ee542e8a599c288d5d0c)

is a 4 × 6 matrix (24 entries) with 8 nonzero elements, so

V         = \[ 10 20 30 40 50 60 70 80 \]
COL\_INDEX = \[  0  1  1  3  2  3  4  5 \]   
ROW\_INDEX = \[  0  2  4  7  8 \]

The whole is stored as 21 entries: 8 in V, 8 in COL\_INDEX, and 5 in ROW\_INDEX.

*   ROW\_INDEX splits the array V into rows: `(10, 20) (30, 40) (50, 60, 70) (80)`, indicating the index of V (and COL\_INDEX) where each row starts and ends;
*   COL\_INDEX aligns values in columns: `(10, 20, ...) (0, 30, 0, 40, ...)(0, 0, 50, 60, 70, 0) (0, 0, 0, 0, 0, 80)`.

Note that in this format, the first value of ROW\_INDEX is always zero and the last is always NNZ, so they are in some sense redundant (although in programming languages where the array length needs to be explicitly stored, NNZ would not be redundant). Nonetheless, this does avoid the need to handle an exceptional case when computing the length of each row, as it guarantees the formula ROW\_INDEX\[_i_ + 1\] − ROW\_INDEX\[_i_\] works for any row _i_. Moreover, the memory cost of this redundant storage is likely insignificant for a sufficiently large matrix.

The (old and new) Yale sparse matrix formats are instances of the CSR scheme. The old Yale format works exactly as described above, with three arrays; the new format combines ROW\_INDEX and COL\_INDEX into a single array and handles the diagonal of the matrix separately.[\[9\]](#cite_note-9)

For [logical](/wiki/Logical_matrix "Logical matrix") [adjacency matrices](/wiki/Adjacency_matrix "Adjacency matrix"), the data array can be omitted, as the existence of an entry in the row array is sufficient to model a binary adjacency relation.

It is likely known as the Yale format because it was proposed in the 1977 Yale Sparse Matrix Package report from Department of Computer Science at Yale University.[\[10\]](#cite_note-10)

In [2]:
import numpy as np
from scipy.sparse import csr_matrix

In [4]:
# Lets look at the official class documentation
print(help(csr_matrix))

Help on class csr_matrix in module scipy.sparse._csr:

class csr_matrix(scipy.sparse._matrix.spmatrix, _csr_base)
 |  csr_matrix(arg1, shape=None, dtype=None, copy=False)
 |
 |  Compressed Sparse Row matrix.
 |
 |  This can be instantiated in several ways:
 |      csr_matrix(D)
 |          where D is a 2-D ndarray
 |
 |      csr_matrix(S)
 |          with another sparse array or matrix S (equivalent to S.tocsr())
 |
 |      csr_matrix((M, N), [dtype])
 |          to construct an empty matrix with shape (M, N)
 |          dtype is optional, defaulting to dtype='d'.
 |
 |      csr_matrix((data, (row_ind, col_ind)), [shape=(M, N)])
 |          where ``data``, ``row_ind`` and ``col_ind`` satisfy the
 |          relationship ``a[row_ind[k], col_ind[k]] = data[k]``.
 |
 |      csr_matrix((data, indices, indptr), [shape=(M, N)])
 |          is the standard CSR representation where the column indices for
 |          row i are stored in ``indices[indptr[i]:indptr[i+1]]`` and their
 |          c

In [7]:
# converting a dense matrix into csr matrix
dense_matrix = np.array([[0, 0, 1], [1, 0, 0], [0, 0, 0]])
csr = csr_matrix(dense_matrix)

In [8]:
csr

<3x3 sparse matrix of type '<class 'numpy.int64'>'
	with 2 stored elements in Compressed Sparse Row format>